Link do dataset >>> link

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Carregando os Dados ---

try:
    train_df = pd.read_csv('https://drive.google.com/uc?export=download&id=1Qhb8AXtSg1s6GtUO_1KemL93r7QYa5Bk')
    test_df = pd.read_csv('https://drive.google.com/uc?export=download&id=1me6rv01pqOK6MBROHQTwTYXCSR_k68gP')
    print("Arquivos 'train.csv' e 'test.csv' carregados com sucesso!")
except FileNotFoundError:
    print("⚠️ Erro: Arquivos 'train.csv' e 'test.csv' não encontrados.")
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()

if not train_df.empty:

    # --- 2. Exploração Inicial (Rápida) ---
    print("\n--- Informações Originais (Treino) ---")
    train_df.info()

    # --- 3. Engenharia de Atributos (Feature Engineering) ---

    # Guardamos os IDs do dataset de teste para a submissão final
    passenger_ids_teste = test_df['PassengerId']

    # Criamos uma lista para aplicar as mesmas transformações em ambos
    data_frames = [train_df, test_df]

    print("\nIniciando Engenharia de Atributos...")

    for df in data_frames:
        # 3.1. Separando o 'PassengerId' (gggg_pp)
        if 'PassengerId' in df.columns:
            df[['Group', 'GroupNumber']] = df['PassengerId'].str.split('_', expand=True)

        # 3.2. Separando a 'Cabin' (deck/num/side)
        if 'Cabin' in df.columns:
            # Preenchemos nulos com um padrão para o split funcionar
            df['Cabin'] = df['Cabin'].fillna('Unknown/0/Unknown')
            df[['Deck', 'CabinNum', 'Side']] = df['Cabin'].str.split('/', expand=True)

    print("Engenharia de Atributos concluída!")


    # --- 4. Definindo Colunas de Gastos ---
    colunas_gastos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']


    # --- 5. Tratamento de Dados Faltantes (NaN) ---
    print("\nIniciando tratamento de dados faltantes (NaN)...")

    # 5.1. Gastos: Nulo (NaN) provavelmente significa gasto 0.
    train_df[colunas_gastos] = train_df[colunas_gastos].fillna(0.0)
    test_df[colunas_gastos] = test_df[colunas_gastos].fillna(0.0)

    # 5.2. Criar feature 'TotalSpend' (Total Gasto)
    # Isso ajuda na lógica do CryoSleep e é uma boa feature
    train_df['TotalSpend'] = train_df[colunas_gastos].sum(axis=1)
    test_df['TotalSpend'] = test_df[colunas_gastos].sum(axis=1)

    # 5.3. CryoSleep (Lógica Inteligente)
    # Se gastou > 0, não pode estar em CryoSleep (False)
    # Se gastou == 0, provavelmente está em CryoSleep (True)
    train_df.loc[(train_df['TotalSpend'] > 0) & (train_df['CryoSleep'].isnull()), 'CryoSleep'] = False
    train_df.loc[(train_df['TotalSpend'] == 0) & (train_df['CryoSleep'].isnull()), 'CryoSleep'] = True

    test_df.loc[(test_df['TotalSpend'] > 0) & (test_df['CryoSleep'].isnull()), 'CryoSleep'] = False
    test_df.loc[(test_df['TotalSpend'] == 0) & (test_df['CryoSleep'].isnull()), 'CryoSleep'] = True

    # 5.4. Age: Preencher com a MEDIANA (mais robusta a outliers)
    # IMPORTANTE: Usamos a mediana *do treino* para preencher ambos
    age_median = train_df['Age'].median()
    print(f"Preenchendo 'Age' (NaN) com a mediana do treino: {age_median}")
    train_df['Age'] = train_df['Age'].fillna(age_median)
    test_df['Age'] = test_df['Age'].fillna(age_median)

    # 5.5. Categóricos (HomePlanet, Destination, VIP): Preencher com a MODA
    # (o valor mais frequente do treino)

    # HomePlanet
    home_planet_mode = train_df['HomePlanet'].mode()[0]
    print(f"Preenchendo 'HomePlanet' (NaN) com a moda do treino: {home_planet_mode}")
    train_df['HomePlanet'] = train_df['HomePlanet'].fillna(home_planet_mode)
    test_df['HomePlanet'] = test_df['HomePlanet'].fillna(home_planet_mode)

    # Destination
    destination_mode = train_df['Destination'].mode()[0]
    print(f"Preenchendo 'Destination' (NaN) com a moda do treino: {destination_mode}")
    train_df['Destination'] = train_df['Destination'].fillna(destination_mode)
    test_df['Destination'] = test_df['Destination'].fillna(destination_mode)

    # VIP
    vip_mode = train_df['VIP'].mode()[0]
    print(f"Preenchendo 'VIP' (NaN) com a moda do treino: {vip_mode}")
    train_df['VIP'] = train_df['VIP'].fillna(vip_mode)
    test_df['VIP'] = test_df['VIP'].fillna(vip_mode)

    print("Tratamento de dados faltantes concluído!")

    # --- 6. Verificação Pós-Tratamento ---
    print("\n--- Resumo dos Dados (Treino) APÓS tratamento de NaN ---")
    # Agora não devemos ter mais valores nulos (exceto 'Transported' no teste)
    train_df.info()

    print("\n--- Resumo dos Dados (Teste) APÓS tratamento de NaN ---")
    test_df.info()

Arquivos 'train.csv' e 'test.csv' carregados com sucesso!

--- Informações Originais (Treino) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB

Iniciando Engenharia de Atributos...
Engenh

/tmp/ipython-input-2556754515.py:102: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_df['VIP'] = train_df['VIP'].fillna(vip_mode)
/tmp/ipython-input-2556754515.py:103: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_df['VIP'] = test_df['VIP'].fillna(vip_mode)


In [ ]:
# --- 7. Codificação de Variáveis Categóricas e Booleanas ---

print("\nIniciando codificação de variáveis...")

# 7.1. Converter colunas Booleanas (True/False) para Inteiros (1/0)
# O modelo entende 1 e 0 melhor do que True/False.
bool_cols = ['CryoSleep', 'VIP']
for col in bool_cols:
    train_df[col] = train_df[col].astype(int)
    test_df[col] = test_df[col].astype(int)

# 7.2. Converter a coluna Alvo (Target)
# Fazemos isso apenas no dataset de treino, pois o de teste não tem essa coluna
if 'Transported' in train_df.columns:
    train_df['Transported'] = train_df['Transported'].astype(int)

# 7.3. Definir colunas categóricas para One-Hot Encoding
# Estas são as colunas de texto que restaram
categorical_cols = ['HomePlanet', 'Destination', 'Deck', 'Side']

# 7.4. Definir colunas para remover
# Não são mais necessárias ou não são úteis para um modelo simples
cols_to_drop = ['PassengerId', 'Name', 'Cabin', 'Group', 'GroupNumber', 'CabinNum']

# Aplicar a remoção
train_df = train_df.drop(columns=cols_to_drop)
test_df = test_df.drop(columns=cols_to_drop)

# 7.5. Aplicar One-Hot Encoding (pd.get_dummies)
# Esta é a forma mais simples de transformar texto em colunas numéricas
# Usamos 'pd.concat' para garantir que treino e teste tenham
# exatamente as mesmas colunas (caso um valor só exista em um dos datasets)

# Salva o número de linhas do treino para podermos separar depois
num_treino = len(train_df)

# Combina os dataframes
combined_df = pd.concat([train_df, test_df], ignore_index=True)

# Aplica o get_dummies no dataframe combinado
combined_df_processed = pd.get_dummies(combined_df, columns=categorical_cols, drop_first=False)

# Separa de volta em treino e teste
train_df_processed = combined_df_processed.iloc[:num_treino]
test_df_processed = combined_df_processed.iloc[num_treino:]

# Importante: O 'Transported' no test_df_processed será NaN,
# mas não tem problema, pois vamos separar X_test e y_test
# Antes de separar, vamos dropar 'Transported' do test_df_processed
# (pois ele não existe lá, só apareceu por causa do concat)
test_df_processed = test_df_processed.drop(columns=['Transported'])
print("Codificação concluída!")

# --- 8. Verificação Pós-Codificação ---
print("\n--- Resumo dos Dados (Treino) APÓS codificação ---")
# Agora TODAS as colunas devem ser numéricas (int ou float)
train_df_processed.info()

print("\n--- Primeiras linhas (Treino) APÓS codificação ---")
# Veja as novas colunas: HomePlanet_Earth, HomePlanet_Europa, etc.
print(train_df_processed.head())


Iniciando codificação de variáveis...
Codificação concluída!

--- Resumo dos Dados (Treino) APÓS codificação ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CryoSleep                  8693 non-null   int64  
 1   Age                        8693 non-null   float64
 2   VIP                        8693 non-null   int64  
 3   RoomService                8693 non-null   float64
 4   FoodCourt                  8693 non-null   float64
 5   ShoppingMall               8693 non-null   float64
 6   Spa                        8693 non-null   float64
 7   VRDeck                     8693 non-null   float64
 8   Transported                8693 non-null   float64
 9   TotalSpend                 8693 non-null   float64
 10  HomePlanet_Earth           8693 non-null   bool   
 11  HomePlanet_Europa          8693 non-null   boo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# --- 9. Preparação Final dos Dados ---

print("\nPreparando dados para o modelo...")

# 9.1. Definir o Alvo (y) e as Features (X) de Treino
# X_train são todas as colunas de treino, MENOS o alvo ('Transported')
X_train = train_df_processed.drop(columns=['Transported'])
# y_train é APENAS a coluna alvo ('Transported')
y_train = train_df_processed['Transported']

# 9.2. Definir o X de Teste
# O X_test já está pronto (ele não tem a coluna 'Transported')
X_test = test_df_processed

print(f"Dimensões de X_train: {X_train.shape}")
print(f"Dimensões de y_train: {y_train.shape}")
print(f"Dimensões de X_test: {X_test.shape}")


# --- 10. Treinamento do Modelo (Árvore de Decisão) ---

print("\nTreinando o modelo DecisionTreeClassifier...")

# 10.1. Criar a instância do modelo
# 'random_state=42' garante que o resultado seja o mesmo toda vez que rodar
model = DecisionTreeClassifier(random_state=42)

# 10.2. Treinar o modelo com TODOS os dados de treino
model.fit(X_train, y_train)

print("Modelo treinado com sucesso!")


# --- 11. Fazer Previsões e Criar Submissão ---
print("\nGerando previsões para o arquivo de submissão...")
# 11.1. Fazer previsões no dataset de teste
predictions_int = model.predict(X_test)

# 11.2. Converter as previsões de volta para o formato (True/False)
# O modelo prevê 1 (True) e 0 (False), mas o Kaggle quer o texto.
predictions_bool = predictions_int.astype(bool)

# 11.3. Criar o DataFrame de submissão
# Usamos os 'passenger_ids_teste' que guardamos lá no começo
submission_df = pd.DataFrame({
    'PassengerId': passenger_ids_teste,
    'Transported': predictions_bool
})

# 11.4. Salvar o arquivo .csv
submission_df.to_csv('submission.csv', index=False)

print("\nArquivo 'submission.csv' criado com sucesso!")
print("Você já pode encontrá-lo no menu lateral (📁) e fazer o download.")

# --- (Opcional) Verificando o arquivo ---
print("\n--- Primeiras linhas do 'submission.csv' ---")
print(submission_df.head())


Preparando dados para o modelo...
Dimensões de X_train: (8693, 27)
Dimensões de y_train: (8693,)
Dimensões de X_test: (4277, 27)

Treinando o modelo DecisionTreeClassifier...
Modelo treinado com sucesso!

Gerando previsões para o arquivo de submissão...

Arquivo 'submission.csv' criado com sucesso!
Você já pode encontrá-lo no menu lateral (📁) e fazer o download.

--- Primeiras linhas do 'submission.csv' ---
  PassengerId  Transported
0     0013_01         True
1     0018_01        False
2     0019_01         True
3     0021_01         True
4     0023_01        False
